In [1]:
workDir = '/home/chantal/Chazy/Chazy_SIP/data/'
figDir = '/home/chantal/Chazy/Chazy_SIP/data/figs/'

physeqDir = '/home/chantal/Chazy/data/phyloseq/'
physeqBulk = 'bulk-core'
physeqBulkSIP = 'bulkSIP-core'


# misc
nprocs = 20

In [2]:
import os
%load_ext rpy2.ipython
#%load_ext pushnote

In [3]:
%%R
library(DESeq2)
library(phyloseq)
library(grid); library(gridExtra)
library(plyr); library(dplyr)
library(tidyr)
library(ggplot2)
library(doParallel)
library(foreach)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: S4Vectors

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: stats4

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: BiocGenerics

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: parallel

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘BiocGenerics’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/funct

In [4]:
if not os.path.isdir(workDir):
    os.makedirs(workDir)
    
if not os.path.isdir(figDir):
    os.makedirs(figDir)    

In [5]:
%%R -i workDir -i figDir -i physeqDir -i physeqBulk -i physeqBulkSIP  -i nprocs 
cat('variables loaded into R\n')

variables loaded into R


## Following Response Groups

In [6]:
%%R
#Response groups
df.Cell = read.table('data/l2fc_sparse-maxRejH_FuncG_cellulose.txt', sep = '\t', header = TRUE)
df.Xyl = read.table('data/l2fc_sparse-maxRejH_FuncG_xylose.txt', sep = '\t', header = TRUE)

In [11]:
%%R
df.Cell.g3 = filter(df.Cell, func_group == 3 )
df.Cell.g24810 = filter(df.Cell, func_group %in% c(2,4,8,10))

df.Xyl.g589 = filter(df.Xyl, func_group %in% c(5,8,9))
df.Xyl.g7 = filter(df.Xyl, func_group == 7 )
df.Xyl.g3 = filter(df.Xyl, func_group == 3)


In [93]:
%%R 
## In Sept Bulk Soils
F = file.path(physeqDir, physeqBulk)
physeq.Bulk = readRDS(F)
physeq.Bulk.m = physeq.Bulk %>% sample_data
physeq.Bulk = prune_samples((physeq.Bulk.m$Sample_Date == '9/24/14') & 
                            (physeq.Bulk.m$Land_Management != 'H2O'),
                           physeq.Bulk)

physeq.Bulk

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 21374 taxa and 16 samples ]
sample_data() Sample Data:       [ 16 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 21374 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 21374 tips and 21373 internal nodes ]


In [94]:
%%R
physeq.list = list(df.Cell.g3, df.Cell.g24810, df.Xyl.g589, df.Xyl.g7, df.Xyl.g3)

In [95]:
%%R
get_l2fc = function(data.set, physeq.obj) {
  
    physeq.Bulk.m = physeq.obj %>% sample_data
    physeq.obj = prune_taxa(as.character(data.set$OTU),
                           physeq.obj) 

    dds = phyloseq_to_deseq2(physeq.obj , ~ Till)
    dds = DESeq(dds, quiet = TRUE, fitType = "local")
        
    theta = 1
    r = results(dds, independentFiltering = FALSE)
    r$OTU = rownames(r)
   
    d = data.frame(r[, c("OTU","log2FoldChange", "lfcSE", "padj")])
    d$padj2 = p.adjust(d$padj, method = "BH")

   # d$padj = p.adjust(p, method = "BH")
    TT = data.frame(tax_table(physeq.obj ))
    TT$OTU = rownames(TT)
    d = left_join(d, TT)
    d$treat = unique(data.set$Treatment)
    d$func_group = head(data.set$func_group, n = 1)
    
    return(d)
}

In [96]:
%%R
df.l2fc = ldply(physeq.list, get_l2fc, physeq.obj = physeq.Bulk)

In [100]:
%%R
df.l2fc %>% group_by(treat, func_group, Rank2, log2FoldChange) %>% filter(padj2 <= 0.10) %>% summarise(n())

Source: local data frame [20 x 5]
Groups: treat, func_group, Rank2 [?]

    treat func_group                    Rank2 log2FoldChange   n()
   (fctr)      (int)                   (fctr)          (dbl) (int)
1     13C          2            __Chloroflexi     -2.2501109     1
2     13C          2         __Proteobacteria      4.6723812     1
3     13C          2         __Proteobacteria      5.1747635     1
4     13C          3         __Actinobacteria     -2.4672878     1
5     13C          3         __Actinobacteria     -1.6784218     1
6     13C          3         __Actinobacteria     -0.9908382     1
7     13C          3         __Actinobacteria      3.0032488     1
8     13C          3 __Candidate_division_WS3      2.5230075     1
9     13C          3            __Chloroflexi      2.1487693     1
10    13C          3            __Chloroflexi      2.6590150     1
11    13C          3            __Chloroflexi      3.3827988     1
12    13C          3            __Chloroflexi      3.3932

## In BulkSIP Soil

In [108]:
%%R 
F = file.path(physeqDir, physeqBulkSIP)
physeq.BulkSIP = readRDS(F)
physeq.BulkSIP.m = physeq.BulkSIP %>% sample_data
physeq.BulkSIP = prune_samples((physeq.BulkSIP.m$Treatment != 'H2O'),
                           physeq.BulkSIP)


physeq.BulkSIP

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 16187 taxa and 104 samples ]
sample_data() Sample Data:       [ 104 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 16187 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 16187 tips and 16186 internal nodes ]


In [109]:
%%R
get_l2fc = function(data.set, physeq.obj) {
  
    physeq.Bulk.m = physeq.obj %>% sample_data
    physeq.obj = prune_taxa(as.character(data.set$OTU),
                           physeq.obj) 

    dds = phyloseq_to_deseq2(physeq.obj , ~ Till)
    dds = DESeq(dds, quiet = TRUE, fitType = "local")
        
    theta = 1
    r = results(dds, independentFiltering = FALSE)
    r$OTU = rownames(r)
   
    d = data.frame(r[, c("OTU","log2FoldChange", "lfcSE", "padj")])
    d$padj2 = p.adjust(d$padj, method = "BH")

   # d$padj = p.adjust(p, method = "BH")
    TT = data.frame(tax_table(physeq.obj ))
    TT$OTU = rownames(TT)
    d = left_join(d, TT)
    d$treat = unique(data.set$Treatment)
    d$func_group = head(data.set$func_group, n = 1)
    
    return(d)
}

In [117]:
%%R
df.l2fc.BulkSIP = ldply(physeq.list, get_l2fc, physeq.obj = physeq.BulkSIP)


In [118]:
%%R
df.l2fc.BulkSIP$Tillage[df.l2fc.BulkSIP$log2FoldChange < 0] = "No-till"
df.l2fc.BulkSIP$Tillage[df.l2fc.BulkSIP$log2FoldChange > 0] = "Till"

In [120]:
%%R
df.l2fc.BulkSIP %>% group_by(treat, func_group, Tillage) %>% filter(padj2 <= 0.10) %>% summarise(n())

Source: local data frame [10 x 4]
Groups: treat, func_group [?]

    treat func_group Tillage   n()
   (fctr)      (int)   (chr) (int)
1     13C          2 No-till    89
2     13C          2    Till   100
3     13C          3 No-till    18
4     13C          3    Till    68
5     13X          3 No-till     4
6     13X          3    Till    30
7     13X          5 No-till    45
8     13X          5    Till    41
9     13X          7 No-till    14
10    13X          7    Till    28
